### Part A

#### Importing Modules

In [241]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
from selenium import webdriver
# from requests_html import HTMLSession

#### Establishing required data

In [201]:
base_url = 'https://www.amazon.in'
url = 'https://www.amazon.in/s?k=bags&crid=2M096C61O4MLT&qid=1653308124&sprefix=ba%2Caps%2C283&ref=sr_pg_1'
header = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Safari/537.36', 'Accept-Language': 'en-US, en;q=0.5'}
# s = HTMLSession()



In [202]:
webpage = requests.get(url,headers=header)
soup = BeautifulSoup(webpage.content, "html.parser")

In [203]:
items = soup.find_all('div',{'class':'a-section a-spacing-small a-spacing-top-small'})

#### Function to get items from all 20 pages

In [204]:
def get_items(url):
    new_url = url
    items = []
    for i in range(20):
        webpage = requests.get(new_url,headers=header)
        soup = BeautifulSoup(webpage.content, "html.parser")
        items1 = soup.find_all('div',{'class':'a-section a-spacing-small a-spacing-top-small'})
        items.extend(items1.copy())
        a_tags = soup.find_all('a')
        for a_tag in a_tags:
            if a_tag.text.strip() == 'Next':
                next_button = a_tag
                new_url = base_url + next_button.get('href')
                break
    return items

In [205]:
items = get_items(url)

In [206]:
len(items)

446

#### Function to get Titles

In [207]:
def get_titles(items):
    titles = []
    for item in items:
        title_tag = item.find('span',{'class':'a-size-medium a-color-base a-text-normal'})
        if title_tag != None:
            title = title_tag.text.strip()
        else:
            title = None
        titles.append(title)
    return titles

In [208]:
titles = get_titles(items)

In [211]:
len(titles)

446

In [ ]:
titles

#### Function to get Links

In [212]:
def get_links(items):
    links = []
    for item in items:
        a_tag = item.find('a',{'class':'a-link-normal s-underline-text s-underline-link-text s-link-style a-text-normal'})
        if a_tag != None:
            link = base_url+a_tag.get('href')
        else:
            link = None
        links.append(link)
    return links

In [213]:
links = get_links(items)

In [214]:
len(links)

446

In [ ]:
links

#### function to get the price of items

In [218]:
def get_price(items):
    prices = []
    for item in items:
        price_tag = item.find('span',{'class':'a-price-whole'})
        if price_tag != None:
            price = price_tag.text
            price = price.replace(',','')
            price = float(price)  
        else:
            price = None
        prices.append(price)    
    return prices

In [220]:
prices = get_price(items)

In [221]:
len(prices)

446

In [ ]:
prices

#### Function to get Ratings

In [223]:
def get_rating(items):
    ratings = []
    for item in items:
        rate_tag = item.find('span',{'class':'a-icon-alt'})
        if rate_tag != None:
            rate = rate_tag.text
            rate = rate[:2]
            rate = float(rate)
        else:
            rate = None
        ratings.append(rate)
    return ratings

In [224]:
ratings = get_rating(items)

In [225]:
len(ratings)

446

In [ ]:
ratings

#### Function to get No. of reviews

In [227]:
def get_review(items):
    reviews = []
    for item in items:
        review_tag = item.find('span',{'class':'a-size-base s-underline-text'})
        if review_tag != None:
            review = review_tag.text
            review = review.replace(',','')
            review = int(review)
        else:
            review = None
        reviews.append(review)
    return reviews

In [228]:
reviews = get_review(items)

In [229]:
len(reviews)

446

In [ ]:
reviews

#### Dataframe

In [231]:
df = pd.DataFrame({'Title':titles,
                   'Price':prices,
                   'Rating(out of 5)':ratings,
                   'No. of Reviews':reviews,
                   'Product Link':links
                   })

In [232]:
df.dropna(inplace=True)

In [233]:
df

,Title,Price,Rating(out of 5),No. of Reviews,Product Link
1,"Swiss Military Hard Shell Travel Backpack UFO,...",3899.0,2.0,1.0,https://www.amazon.in/sspa/click?ie=UTF8&spc=M...
2,Elegant 16 inch 28 L Casual Waterproof Laptop ...,799.0,5.0,3.0,https://www.amazon.in/sspa/click?ie=UTF8&spc=M...
3,Wesley Milestone 2.0 Casual Waterproof Laptop ...,570.0,4.0,11063.0,https://www.amazon.in/Wesley-Milestone-Waterpr...
4,American Tourister 32 Ltrs Black Casual Backpa...,1199.0,4.0,54420.0,https://www.amazon.in/American-Tourister-AMT-S...
5,Skybags Brat Black 46 Cms Casual Backpack,619.0,4.0,4628.0,https://www.amazon.in/Skybags-Brat-Black-Casua...
...,...,...,...,...,...
441,American Tourister Amt Rexton 01 Navy Lightwei...,2049.0,4.0,63.0,https://www.amazon.in/American-Tourister-Light...
442,MOKOBARA The Transit Backpack Laptop Bag for M...,4999.0,4.0,610.0,https://www.amazon.in/sspa/click?ie=UTF8&spc=M...
443,Gear DOUBLE DECKER 31L Water Resistant Laptop ...,1189.0,3.0,6.0,https://www.amazon.in/sspa/click?ie=UTF8&spc=M...
444,Gear DOUBLE DECKER 31L Water Resistant Laptop ...,1399.0,3.0,4.0,https://www.amazon.in/sspa/click?ie=UTF8&spc=M...


### Part B

In [234]:
df['ASIN'] = None
df['Manufacturer'] = None
df['Description'] = None

In [313]:
ASINs = []
manufacturers = []
descriptions = []

In [238]:
df

,Title,Price,Rating(out of 5),No. of Reviews,Product Link,ASIN,Manufacturer,Description
1,"Swiss Military Hard Shell Travel Backpack UFO,...",3899.0,2.0,1.0,https://www.amazon.in/sspa/click?ie=UTF8&spc=M...,None,None,None
2,Elegant 16 inch 28 L Casual Waterproof Laptop ...,799.0,5.0,3.0,https://www.amazon.in/sspa/click?ie=UTF8&spc=M...,None,None,None
3,Wesley Milestone 2.0 Casual Waterproof Laptop ...,570.0,4.0,11063.0,https://www.amazon.in/Wesley-Milestone-Waterpr...,None,None,None
4,American Tourister 32 Ltrs Black Casual Backpa...,1199.0,4.0,54420.0,https://www.amazon.in/American-Tourister-AMT-S...,None,None,None
5,Skybags Brat Black 46 Cms Casual Backpack,619.0,4.0,4628.0,https://www.amazon.in/Skybags-Brat-Black-Casua...,None,None,None
...,...,...,...,...,...,...,...,...
441,American Tourister Amt Rexton 01 Navy Lightwei...,2049.0,4.0,63.0,https://www.amazon.in/American-Tourister-Light...,None,None,None
442,MOKOBARA The Transit Backpack Laptop Bag for M...,4999.0,4.0,610.0,https://www.amazon.in/sspa/click?ie=UTF8&spc=M...,None,None,None
443,Gear DOUBLE DECKER 31L Water Resistant Laptop ...,1189.0,3.0,6.0,https://www.amazon.in/sspa/click?ie=UTF8&spc=M...,None,None,None
444,Gear DOUBLE DECKER 31L Water Resistant Laptop ...,1399.0,3.0,4.0,https://www.amazon.in/sspa/click?ie=UTF8&spc=M...,None,None,None


#### Function to get ASIN

In [236]:
def get_asin(page):
    tag = None
    tags = page.find_all('th',{'class':'a-color-secondary a-size-base prodDetSectionEntry'})
    for t in tags:
        if 'ASIN' in t.text:
            tag = t.parent
            break
    if tag != None:
        tag = tag.find('td')
        return tag.text
    tags = page.find_all('span',{'class':'a-text-bold'})
    for t in tags:
        if 'ASIN' in t.text:
            tag = t
            break
    if tag != None:
        tag = tag.parent
        tags = tag.find_all('span')
        if tags[2] != None:
            return tags[2].text
    return None


#### Function to get Manufacturer

In [237]:
def get_manufacturer(page):
    tag = None
    tags = page.find_all('th',{'class':'a-color-secondary a-size-base prodDetSectionEntry'})
    for t in tags:
        if 'Manufacturer' in t.text:
            tag = t.parent
            break
    if tag != None:
        tag = tag.find('td')
        return tag.text
    tags = page.find_all('span',{'class':'a-text-bold'})
    for t in tags:
        if 'manufacturer' in t.text:
            tag = t.parent
            break
    if tag != None:
        tag = tag.find_all('span')
        if tag[2] != None:
            return tag[2].text
    return None

    tag = tag.parent

Note: Could not access the product pages

In [170]:

for index,row in df.iterrows():
    webpage = requests.get(row['Product Link'])
    soup = BeautifulSoup(webpage.content, "html.parser")
    ASINs.append(get_asin(soup))
    manufacturers.append(get_manufacturer(soup))
    

IndexError: list index out of range

Checking why product page could not be accessed

In [242]:
url = 'https://www.amazon.in/American-Tourister-AMT-SCH-02/dp/B07CJCGM1M'
page = requests.get(url,headers=header)
soup = BeautifulSoup(webpage.content, "html.parser")

Amazon returning CAPTCHA page instead of actual product page

In [198]:
soup

<!DOCTYPE html>

<!--[if lt IE 7]> <html lang="en-us" class="a-no-js a-lt-ie9 a-lt-ie8 a-lt-ie7"> <![endif]-->
<!--[if IE 7]>    <html lang="en-us" class="a-no-js a-lt-ie9 a-lt-ie8"> <![endif]-->
<!--[if IE 8]>    <html lang="en-us" class="a-no-js a-lt-ie9"> <![endif]-->
<!--[if gt IE 8]><!-->
<html class="a-no-js" lang="en-us"><!--<![endif]--><head>
<meta content="text/html; charset=utf-8" http-equiv="content-type"/>
<meta charset="utf-8"/>
<meta content="IE=edge,chrome=1" http-equiv="X-UA-Compatible"/>
<title dir="ltr">Amazon.in</title>
<meta content="width=device-width" name="viewport"/>
<link href="https://images-na.ssl-images-amazon.com/images/G/01/AUIClients/AmazonUI-3c913031596ca78a3768f4e934b1cc02ce238101.secure.min._V1_.css" rel="stylesheet"/>
<script>

if (true === true) {
    var ue_t0 = (+ new Date()),
        ue_csm = window,
        ue = { t0: ue_t0, d: function() { return (+new Date() - ue_t0); } },
        ue_furl = "fls-eu.amazon.in",
        ue_mid = "A21TJRUUN4KGV",


#### Using Chrome Driver

In [285]:
driver_path = 'C:/Users/Lenovo/Downloads/chromedriver_win32/chromedriver.exe'

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_27056\439724934.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome(executable_path=driver_path)


#### Function to get ASIN

In [280]:
def get_asin():
    tag = None
    th = None
    span = None
    TH = browser.find_elements('xpath',"//th[@class='a-color-secondary a-size-base prodDetSectionEntry']")
    for t in TH:
        if 'ASIN' in t.text:
            th = t
            break
    if th != None:
        tag = th.find_element('xpath','./following-sibling::td')
        if tag!=None:
            return tag.text
    spans = browser.find_elements('xpath',"//span[@class='a-text-bold']")
    for s in spans:
        if 'ASIN' in s.text:
            span = s
            break
    if span!= None:
        tag = span.find_element('xpath','./following-sibling::span')
        if tag!=None:
            return tag.text
    return None
    

#### Function to get Manufcaturer details

In [283]:
def get_manufacturer():
    tag = None
    th = None
    span = None
    TH = browser.find_elements('xpath',"//th[@class='a-color-secondary a-size-base prodDetSectionEntry']")
    for t in TH:
        if 'Manufacturer' in t.text:
            th = t
            break
    if th != None:
        tag = th.find_element('xpath','./following-sibling::td')
        if tag!=None:
            return tag.text
    spans = browser.find_elements('xpath',"//span[@class='a-text-bold']")
    for s in spans:
        if 'Manufacturer' in s.text:
            span = s
            break
    if span!= None:
        tag = span.find_element('xpath','./following-sibling::span')
        if tag!=None:
            return tag.text
    return None

#### Function to get Product Description

In [305]:
def get_description():
    tag = None
    tag = browser.find_element('xpath',"//div[@id='feature-bullets']")
    if tag == None:
        return None
    text1 = tag.text.replace('\n','')
    text1 = text1.replace('. ','.\n')
    return text1

#### Extracting details of each product

In [316]:
i = 1
rows = len(df)
for index,row in df.iterrows():
    link = row['Product Link']
    browser = webdriver.Chrome(executable_path=driver_path)
    browser.get(link)
    ASINs.append(get_asin())
    manufacturers.append(get_manufacturer())
    descriptions.append(get_description())
    browser.close()
    print(f'{i}/{rows} Done.')
    i+=1
    

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_27056\513585967.py:5: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome(executable_path=driver_path)


1/415 Done.
2/415 Done.
3/415 Done.
4/415 Done.
5/415 Done.
6/415 Done.
7/415 Done.
8/415 Done.
9/415 Done.
10/415 Done.
11/415 Done.
12/415 Done.
13/415 Done.
14/415 Done.
15/415 Done.
16/415 Done.
17/415 Done.
18/415 Done.
19/415 Done.
20/415 Done.
21/415 Done.
22/415 Done.
23/415 Done.
24/415 Done.
25/415 Done.
26/415 Done.
27/415 Done.
28/415 Done.
29/415 Done.
30/415 Done.
31/415 Done.
32/415 Done.
33/415 Done.
34/415 Done.
35/415 Done.
36/415 Done.
37/415 Done.
38/415 Done.
39/415 Done.
40/415 Done.
41/415 Done.
42/415 Done.
43/415 Done.
44/415 Done.
45/415 Done.
46/415 Done.
47/415 Done.
48/415 Done.
49/415 Done.
50/415 Done.
51/415 Done.
52/415 Done.
53/415 Done.
54/415 Done.
55/415 Done.
56/415 Done.
57/415 Done.
58/415 Done.
59/415 Done.
60/415 Done.
61/415 Done.
62/415 Done.
63/415 Done.
64/415 Done.
65/415 Done.
66/415 Done.
67/415 Done.
68/415 Done.
69/415 Done.
70/415 Done.
71/415 Done.
72/415 Done.
73/415 Done.
74/415 Done.
75/415 Done.
76/415 Done.
77/415 Done.
78/415 D

In [317]:
df['ASIN'] = ASINs
df['Manufacturer'] = manufacturers
df['Description'] = descriptions

In [318]:
df

,Title,Price,Rating(out of 5),No. of Reviews,Product Link,ASIN,Manufacturer,Description
1,"Swiss Military Hard Shell Travel Backpack UFO,...",3899.0,2.0,1.0,https://www.amazon.in/sspa/click?ie=UTF8&spc=M...,B0C3MHG797,"Swiss Military Consumer Goods Limited, W-39, O...",Care Instructions: Wipe with Dry ClothUnique D...
2,Elegant 16 inch 28 L Casual Waterproof Laptop ...,799.0,5.0,3.0,https://www.amazon.in/sspa/click?ie=UTF8&spc=M...,B0C4GWK9HR,None,Dimensions and Capacity]: (48 Cm X 33 Cm X 28 ...
3,Wesley Milestone 2.0 Casual Waterproof Laptop ...,570.0,4.0,11063.0,https://www.amazon.in/Wesley-Milestone-Waterpr...,B084JGJ8PF,"Wesley, Longani Trading Company, F-82, Sector-...",30L Capacity: The Backpack has a padded laptop...
4,American Tourister 32 Ltrs Black Casual Backpa...,1199.0,4.0,54420.0,https://www.amazon.in/American-Tourister-AMT-S...,B07CJCGM1M,Samsonite,"Laptop Compatibility: No, Strap Type: Adjustab..."
5,Skybags Brat Black 46 Cms Casual Backpack,619.0,4.0,4628.0,https://www.amazon.in/Skybags-Brat-Black-Casua...,B08Z1HHHTD,VIP Industries Ltd,Combination of functional & safety features in...
...,...,...,...,...,...,...,...,...
441,American Tourister Amt Rexton 01 Navy Lightwei...,2049.0,4.0,63.0,https://www.amazon.in/American-Tourister-Light...,B08425H5CV,None,"Inside Slip in pocket for accessories, Smart s..."
442,MOKOBARA The Transit Backpack Laptop Bag for M...,4999.0,4.0,610.0,https://www.amazon.in/sspa/click?ie=UTF8&spc=M...,B09V7R98XC,Designed By Mokobara Lifestyle Private Limited...,Care Instructions: Wipe with Damp ClothCRAFTED...
443,Gear DOUBLE DECKER 31L Water Resistant Laptop ...,1189.0,3.0,6.0,https://www.amazon.in/sspa/click?ie=UTF8&spc=M...,B0BTJ3WTMV,Gear Merchandise Pvt Ltd,Ergonomic Shape : Spacious Compartments : Intu...
444,Gear DOUBLE DECKER 31L Water Resistant Laptop ...,1399.0,3.0,4.0,https://www.amazon.in/sspa/click?ie=UTF8&spc=M...,B0BTJ2T26W,Gear Merchandise Pvt Ltd,Ergonomic Shape : Spacious Compartments : Intu...


#### Exporting to CSV

In [319]:
df.to_csv('Amazon_Bags.csv', index=False)